# Chapter 6 - Integrity and consistency in the Domain

Last chapter we talked about creating types that model the domain accurately.  
Here we want to look at constructing a boundary within which we can totally trust that all data is valid and in this way we can avoid coding defensively within this boundary.  
In this chapter we talk about 2 aspects of a trusted domain, integrity and consistency
## Integrity
The piece of data follows the correct business rules  
For example
- A `UnitQuantity` must be between 1 and 1000.
- An `Order` must have at least 1 `OrderLine`
- An `Order` must have a validated shipping address before being sent to the shipping department

## Consistency
Different parts of the domain model must agree about facts.
For example:
- The total amount to bill for an order must be the total of the individual lines.
- When an order is placed, a corresponding invoice must be created.
- If a discount voucher is used, it must be marked as used so it can't be used again.

### Integrity for simple values
- We saw that we should not represent simple values with primitives, but by domain focussed types such as WidgetCode or UnitQuantity.
- In real life it's rare to have an unbounded integer or string.
- We started by creating UnitQuantity as a wrapped int value, but we can constrain it further.


In [1]:
module Domain =
    type UnitQuantity = private UnitQuantity of int

    module UnitQuantity =
        let create qty =
            if qty < 1 then
                Error "UnitQuantity cannot be negative"
            else if qty > 1000 then
                Error "UnitQuantity cannot be more than 1000"
            else
                Ok (UnitQuantity qty)

        let value (UnitQuantity qty) = qty

        let (|UnitQty|) (UnitQuantity uQty) = uQty

In [27]:
open Domain

let uQty = UnitQuantity.create 100

let print1 = function
    | Ok qty -> printfn "%d" (UnitQuantity.value qty)
    | Error msg -> printfn "%s" msg

let print2 = function
    | Ok (UnitQuantity.UnitQty x) -> printfn "%d" x
    | Error msg -> printfn "%s" msg


uQty
|> print2

100


#### Units of measure
- For numeric values, we can additionally document requirements with units of measure.
- Units of measure are annotations for units of a number.

In [17]:
[<Measure>]
type kg // kilogram

[<Measure>]
type m // metre


You can define units in terms of other measures too

In [18]:
[<Measure>]
type cm // centimetre

// A millilitre is the same as a cubic centimetre
[<Measure>]
type ml = cm^3

In [2]:
open FSharp.Data.UnitSystems.SI.UnitSymbols // SI units are defined in this module

type OrderQuantity =
    | UnitQuantity of int
    | KilogramQuantity of decimal<kg>

let printQuantity = function
    | UnitQuantity qty ->
        printfn "%i units" qty
    | KilogramQuantity kgQty ->
        printfn "%g kg" kgQty

printQuantity (KilogramQuantity 1.0M<kg>)
printQuantity (UnitQuantity 1)



1 kg
1 units


In [3]:
open FSharp.Data.UnitSystems.SI.UnitSymbols // SI units are defined in this module

[<Measure>]
type g

// The compiler will enforce compatibility between units
let convertg2kg (x : float<g>): float<kg> = x / 1000.0<g/kg>

let fiveKilos = 5.0<kg>
let fiveMetres = 5.0<m>

// (fiveKilos = fiveMetres) |> ignore
// type mismatch between float<kg> and float<m>

// let listOfWeights = [
//     fiveKilos
//     fiveMetres
// ]
// type mismatch between float<kg> and float<m>

// Units of measure can also be generic
let genericSumUnits<[<Measure>] 'u> (x: float<'u>) (y: float<'u>) = x + y

type Vector3D<[<Measure>] 'u> = {
    x: float<'u>
    y: float<'u>
    z: float<'u>
    }

Units of measure have no overhead because they are only used by the compiler and are erased in the compiled code 🎉

## Enforcing invariants in the type system
- An invariant is a condition that stays true no matter what else happens.
- A UnitQuantity must be between 1 and 1000. That is an example of an invariant.
- Another example of an invariant is that an order must have at least one line. This is an invariant we can enforce directly in the type system.

In [21]:
type NonEmptyList<'a> = {
    First: 'a
    Rest: 'a list
    }

module NonEmptyList =
    let nonEmptyList = {
        First = 1
        Rest = [2; 3; 4]
    }

    let listWithOneElement = {
        First = 1
        Rest = []
    }

    let head = _.First
    let tail = _.Rest
    let add item list = {
        First = item
        Rest = list.First :: list.Rest
    }

// There is no way to create a list with no elements

## Capturing business rules in the type system
- If we decide some business rules should depend on whether an email address is verified or not, we can capture this in the type system.

In [4]:
type EmailAddress = EmailAddress of string

module Naive =
    type CustomerEmail = {
        EmailAddress : EmailAddress
        IsVerified: bool
        }


- The problem with this approach is it can cause inconsistency. 
- The IsVerified flag is not necessarily dependant on whether the email is verified or not.
- There could be a security breach if the IsVerified flag gets set to true without the email actually being verified.
- Alternatively a developer could forget to set the flag in the verification workflow.

In [23]:
module Better =
    type CustomerEmail =
        | Unverified of EmailAddress
        | Verified of EmailAddress

- This doesn't prevent us from creating a Verified case by passing in an unverified EmailAddress.

In [24]:
module Best =
    type VerifiedEmailAddress = private VerifiedEmailAddress of string

    type CustomerEmail =
        | Unverified of EmailAddress
        | Verified of VerifiedEmailAddress

    type VerifyEmailFlow = EmailAddress -> VerifiedEmailAddress

    type SendPasswordResetEmail = VerifiedEmailAddress -> Async<unit>

- In this module, we create a new type for `VerifiedEmailAddress`. We give it a private constructor so it can only be created in this module.
- We can't create a `Verified` case without having a `VerifiedEmailAddress`, and this module controls the creation of Verified email addresses.
- We can also now explicitly document a workflow that sends a password reset message, to which you can't pass an unverified email address.

## Consistency
- Consistency is a business term rather than a technical one.
- It is always context-dependent. It depends on what the business needs.
- Consistency and atomicity of persistence are linked. We cannot ensure consistency if entities are not persisted atomically.
### Within a single aggregate
- We talked about aggregates and how they act as a consistency boundary last week.
- We want the order's orderlines to add up to the total price
- If we have some derived values, it's easier to recalculate it than to store it. (Probably depends on how expensive the computation is)
- If we need to persist the data, we need to ensure it stays in sync.
- If one of the lines of an order is updated, the total must be updated too to keep it consistent.
- Only the order knows how to preserve consistency. The order is the aggregate that enforces a consistency boundary.

In [25]:
module Aggregate =
    module Order =
        type OrderLine = {
            Id: int
            Price: double
            }

        type Order = private {
            OrderLines: OrderLine list
            AmountToBill: double
            }

        let private findOrderLine orderLineId =
            List.find (_.Id >> ((=) orderLineId))

        let private replaceOrderLine orderLines orderLineId newOrderLine =
            orderLines |> List.map (fun x -> if x.Id = orderLineId then newOrderLine else x)

        let empty = {
            OrderLines = List.empty
            AmountToBill = 0
            }

        let addOrderLine order orderLine =
            let newOrderLines = orderLine :: order.OrderLines
            let newAmountToBill = newOrderLines |> List.sumBy _.Price
            let newOrder = {
                order with
                    OrderLines = newOrderLines
                    AmountToBill = newAmountToBill
                }
            newOrder

        let changeOrderLinePrice order orderLineId newPrice =
            let orderLine = order.OrderLines |> findOrderLine orderLineId
            let newOrderLine = { orderLine with Price = newPrice }
            let newOrderLines =
                replaceOrderLine order.OrderLines orderLineId newOrderLine
            let newAmountToBill = newOrderLines |> List.sumBy _.Price
            let newOrder = {
                order with
                    OrderLines = newOrderLines
                    AmountToBill = newAmountToBill
                }

            newOrder

In [26]:
open Aggregate

let order1 = Order.addOrderLine Order.empty {Id = 0; Price = 2.}
let order2 = Order.changeOrderLinePrice order1 0 3.
printfn "%A" order1
printfn "%A" order2

{ OrderLines = [{ Id = 0
                  Price = 2.0 }]
  AmountToBill = 2.0 }
{ OrderLines = [{ Id = 0
                  Price = 3.0 }]
  AmountToBill = 3.0 }


### Between different contexts
- When the order is placed, a corresponding invoice must be created.
- Invoicing is part of the billing domain. We can't manipulate the objects of another domain because we need to keep the contexts isolated and decoupled
- We need to use the billing context's public API
- There are ways to synchronize updates between contexts, but we usually don't need to do that. We can just send an async message to the context an continue
- What happens if no invoice is created?
	- Do nothing. The customer gets free stuff and the business has to write off the cost.
	- Detect that the message was lost and resend it with a reconciliation process
	- Create a compensating action that undoes the previous action. In this case, cancel the order, but more realistically it would be used to correct mistakes in an order or issue a refund.
- In all cases there is no need for rigid coorditation.
- If we require consistency, we need to implement the second or third option, but the information won't be consistent immediately. These types of systems are 'eventually consistent'.
- Eventual consistency is not optional consistency. It's important that the system is consistent at some point in the future.

### Between aggregates in the same context
- If 2 aggregates need to be consistent with each other, should we update them together in the same transaction, or separately with eventual consistency?
- It depends
- A useful guideline is 'only update one aggregate per transaction'.
- If more than 1 aggregate is involved, we should use messages and eventual consistency.
- Some times if the workflow is considered by the business to be a single transaction, you can update entities in the same transaction
- For example when transferring money between accounts, where 1 account increases and the other decreases.
- If the accounts are represented by an account aggregate, they should be updated in the same transaction to maintain consistency.
- In this case, it might be a clue that you can refactor.
- The transaction often has it's own identity, so it could be an entity of its own.

### Multiple aggregates acting on the same data
- Account aggregate and MoneyTransfer aggregate might both act on account balance, both needing to ensure it doesn't become negative
- Constraints can be shared if they are modelled using types.
- The balance could be modelled with a NonNegativeMoney type